### import

In [59]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from math import exp, sqrt, log
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import i0, i1, iv
from numpy import random
from torch.nn.functional import normalize
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torchsummary import summary

### The setup of the dataset

In [65]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)


### Define the model of the neural network

In [4]:
class BASICnet(nn.Module):
    def __init__(self, dim, width):
        super(BASICnet, self).__init__()
        self.dim = dim
        self.width = width

        self.fc1 = nn.Linear(self.dim, self.width)
        self.fc2 = nn.Linear(self.width, self.width)
        self.fc3 = nn.Linear(self.width, self.width)
        self.fc4 = nn.Linear(self.width, self.width)

        self.output = nn.Linear(self.width, 1, bias=True)

    def forward(self, x):
        # s = nn.ConstantPad2d(x, (0, self.dim - self.dim))
        y = self.fc1(x)
        y = torch.tanh(y)
        y = self.fc2(y)
        y = torch.tanh(y)
        y = self.fc3(y)
        y = torch.tanh(y)
        y = self.fc4(y)
        y = torch.tanh(y)
        
        output = self.output(y)
        return output

    def assign_value(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                nn.init.constant_(m.bias.data, 0.1)

### Cauculate equation (Righthand side of the equation, RHS)

In [5]:
def calculate_fx(x):
    pi = torch.tensor(np.pi, dtype=torch.float32)
    f_x = -160 * pi**2 * torch.prod(torch.sin(4 * pi * x), dim=-1)
    return f_x

### Generate the advice calculate_fx

In [50]:
def calculate_fx(X):
    # Check if there are boundary cases with 1 and -1
    has_boundary = torch.any((X == -1) | (X == 1))

    if has_boundary:
        # If there are boundary cases, set fx to zero
        fx = torch.tensor(0.0, dtype=torch.float32)
    else:
        # Calculate the regular fx
        fx = -160 * (torch.tensor(4 * np.pi, dtype=torch.float32).pow(2)) * torch.prod(torch.sin(4 * np.pi * X), dim=-1)

    return fx


### Generate model from the dataset and using montecarlo method to train the model

In [48]:
import random

def data_generator_monte_carlo(Numberofgenpoints, dim, prob_special = 0.1):
    source = torch.randn(size = (Numberofgenpoints, dim)) # 生成一个大矩阵
    source = normalize(source, p=2)
    radius = torch.rand(size = (Numberofgenpoints, 1))
    radius = torch.pow(torch.rand(size = (Numberofgenpoints, 1)), 1/dim)
    source = source * radius

    # 随即确认替换的行
    num_replace_incol = torch.randint(1, Numberofgenpoints, size=(1,)).item()

    for _ in range(num_replace_incol):
        if random.random() < prob_special:
            special_value = random.choice([1, -1])
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            source[row_index] = torch.full((dim, ), special_value)
        else:
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            col_index = torch.randint(0, dim, size=(1,)).item()
            replace_value = torch.randint(0, 2, size=(1,)).item() * 2 - 1
            source[row_index, col_index] = replace_value

    max_value = torch.max(source)
    min_value = torch.min(source)

    if max_value > 1 or min_value < -1:
        raise ValueError
    return source

已经生成了包括边界条件的值。

### Dataset generate

In [49]:
GenPointsNum = 10000
Dim = 10
source = data_generator_monte_carlo(GenPointsNum, Dim, 0.05)

In [117]:
import torch.nn as nn
import torch.nn.functional as F

class CustomLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(CustomLoss, self).__init__()
        self.weight = weight


    def forward(self, y_pred, X):
        with torch.autograd.set_detect_anomaly(True):
            u = y_pred
            u_x = torch.autograd.grad(u, X, torch.ones_like(u), create_graph=True)[0]
            u_xx = torch.autograd.grad(u_x, X, torch.ones_like(u_x), create_graph=True)[0]
            fx = calculate_fx(X)
            fx = torch.tensor(fx, dtype=torch.float32)

            # 计算 PDE 损失
            pde_loss = F.mse_loss(u_xx.sum(dim=1), fx)

            # 计算边界损失
            boundary_loss = F.mse_loss(u, torch.zeros_like(u))

            # 计算均方差损失(Mean Absolute Error)
            mae = F.l1_loss(u_xx.sum(dim=1), fx)

            # 总损失
            total_loss = pde_loss + boundary_loss + 0.5 * mae

            return total_loss

loss_fn = CustomLoss()




## The optimizer and scheduler

In [115]:
from torch.utils.data import DataLoader

LR = 1e-3
NUM_EPOCHS = 10000
BATCH_SIZE = 1000
loss_list = np.zeros(NUM_EPOCHS)
test_loss = np.zeros(NUM_EPOCHS)
max_value_loss = np.zeros(NUM_EPOCHS)
min_value_loss = np.zeros(NUM_EPOCHS)

net = BASICnet(dim=Dim, width=500)

# Optimizer and scheduler define there
Optimizer = optim.Adam(net.parameters(), lr=LR)
Scheduler = StepLR(Optimizer, step_size=1000, gamma=0.8)

# orignal print
print(sum(p.numel() for p in net.parameters() if p.requires_grad))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
summary(net, (Dim,))

data_loader = DataLoader(source, batch_size=BATCH_SIZE, shuffle=True)

757501
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]           5,500
            Linear-2                  [-1, 500]         250,500
            Linear-3                  [-1, 500]         250,500
            Linear-4                  [-1, 500]         250,500
            Linear-5                    [-1, 1]             501
Total params: 757,501
Trainable params: 757,501
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.89
Estimated Total Size (MB): 2.90
----------------------------------------------------------------


In [118]:

from tqdm import trange

# 其他定义和初始化

for epoch in trange(NUM_EPOCHS):
    total_loss = 0.0
    for batch in data_loader:
        batch = batch.to(device)  # 将数据移到GPU上，如果可用
        Optimizer.zero_grad()
        y_pred = net(batch)
        loss = loss_fn(y_pred, batch)  # 不再传递y_true
        loss.backward()
        Optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}")


  0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: One of the differentiated Tensors does not require grad